## 연속으로 발생되는 이미지 내 Detection에 대해, decision을 하기 위한 과정
- History프레임을 20~30프레임 적당히 설정.
- 각 프레임에서 발생하는 detection좌표를 Queue에 저장
- 현재 발생하는 detection좌표와 History Queue에 기록중 겹치는 것이 있는지 체크
- 겹침이 있으면 detection의 confidence를 누적 confidence에 합쳐줌, 아니면 누적 confidence를 감소시켜 줌
- 누적 confidence가 어떤 값을 초과하면 감지가 발생한 것으로 봄

In [27]:
import cv2
import numpy as np
from collections import deque

# Parameters
ALERT_THRESHOLD = 20  # 특정 영역에서 Box가 나타난 횟수 및 confidence 합계의 임계값
MAX_HISTORY = 30  # 빈도를 추적할 프레임 수
ALERT_WINDOW = 100  # 근방으로 판단할 영역 크기 (픽셀 단위)
CONFIDENCE_THRESHOLD = 0.5  # 감지된 confidence의 임계값
ALERT_CONFIDENCE_THRESHOLD = 5  # 알람을 ON으로 간주하기 위한 최소 confidence 합계

# History to keep track of box occurrences
box_history = deque(maxlen=MAX_HISTORY)
confidence_history = deque(maxlen=MAX_HISTORY)

# Function to detect box (stub function, replace with real detection logic)
def detect_box():
    # 예시: 랜덤한 박스와 confidence를 생성
    h, w = 480, 640 # frame.shape
    box = (np.random.randint(0, w-100), np.random.randint(0, h-100),
           np.random.randint(100, w), np.random.randint(100, h))
    confidence = np.random.random()
    return box, confidence

# 현재 발생 box의 좌표가 Queue내 저장된 box중 하나라도 겹치는 것이 있으면 True리턴
def is_within_alert_area(box, history, alert_window):
    for past_box in history:
        if past_box:
            px1, py1, px2, py2 = past_box
            bx1, by1, bx2, by2 = box
            if (abs(px1 - bx1) < alert_window and abs(py1 - by1) < alert_window and
                abs(px2 - bx2) < alert_window and abs(py2 - by2) < alert_window):
                return True
    return False

# Main video processing loop
#cap = cv2.VideoCapture(0)  # 0번 카메라 사용, 파일 사용 시 "your_video.mp4"로 변경
alarm_on = False
alarm_counter = 0

# while cap.isOpened():
#     ret, frame = cap.read()
#     if not ret:
#         break

for ii in range(100):

    # box와 box의 confidence는 이미지당 1개만 생성됨
    box, confidence = detect_box()
    
    if confidence > CONFIDENCE_THRESHOLD:
        # 기록된 히스토리 중 근방에서의 발생 빈도와 confidence 확인
        # Queue내의 저장영역들과 하나라도 겹침이 있으면 confidence더함(증가)
        if is_within_alert_area(box, box_history, ALERT_WINDOW):
            alarm_counter += confidence  # confidence 값을 누적
        else:  # 아니면 -1 또는 -0.1 등 적당한 값을 감소시킴 
            alarm_counter = max(0, alarm_counter - 0.1)
    else:
        alarm_counter = max(0, alarm_counter - 0.1)

    # History 업데이트
    box_history.append(box)
    confidence_history.append(confidence)

    # 알람 상태 결정
    if alarm_counter > ALERT_CONFIDENCE_THRESHOLD:
        alarm_on = True
    else:
        alarm_on = False

    print(f"프레임: {ii:3d}, 알람 값: {alarm_counter:6.3f}, 알람: {alarm_on}")
    
    # 알람 상태 시각화
    # if alarm_on:
    #     cv2.putText(frame, "ALARM ON", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    # else:
    #     cv2.putText(frame, "ALARM OFF", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # 박스 그리기
    if confidence > CONFIDENCE_THRESHOLD:
        x1, y1, x2, y2 = box
        # print(confidence, box)
        # cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
        # cv2.putText(frame, f"Conf: {confidence:.2f}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

    # # 결과 프레임 출력
    # cv2.imshow('frame', frame)
    
    # if cv2.waitKey(1) & 0xFF == ord('q'):
    #     break

# cap.release()
# cv2.destroyAllWindows()

프레임:   0, 알람 값:  0.000, 알람: False
프레임:   1, 알람 값:  0.000, 알람: False
프레임:   2, 알람 값:  0.000, 알람: False
프레임:   3, 알람 값:  0.000, 알람: False
프레임:   4, 알람 값:  0.000, 알람: False
프레임:   5, 알람 값:  0.000, 알람: False
프레임:   6, 알람 값:  0.000, 알람: False
프레임:   7, 알람 값:  0.000, 알람: False
프레임:   8, 알람 값:  0.000, 알람: False
프레임:   9, 알람 값:  0.999, 알람: False
프레임:  10, 알람 값:  0.899, 알람: False
프레임:  11, 알람 값:  0.799, 알람: False
프레임:  12, 알람 값:  0.699, 알람: False
프레임:  13, 알람 값:  0.599, 알람: False
프레임:  14, 알람 값:  1.384, 알람: False
프레임:  15, 알람 값:  1.284, 알람: False
프레임:  16, 알람 값:  1.184, 알람: False
프레임:  17, 알람 값:  1.084, 알람: False
프레임:  18, 알람 값:  0.984, 알람: False
프레임:  19, 알람 값:  0.884, 알람: False
프레임:  20, 알람 값:  0.784, 알람: False
프레임:  21, 알람 값:  0.684, 알람: False
프레임:  22, 알람 값:  0.584, 알람: False
프레임:  23, 알람 값:  1.548, 알람: False
프레임:  24, 알람 값:  1.448, 알람: False
프레임:  25, 알람 값:  1.348, 알람: False
프레임:  26, 알람 값:  1.248, 알람: False
프레임:  27, 알람 값:  1.148, 알람: False
프레임:  28, 알람 값:  2.125, 알람: False
프레임:  29, 알람 값

In [35]:
import cv2
import numpy as np
from collections import deque

# Parameters
ALERT_THRESHOLD = 20  # 특정 영역에서 Box가 나타난 횟수 및 confidence 합계의 임계값
MAX_HISTORY = 30  # 빈도를 추적할 프레임 수
ALERT_WINDOW = 100  # 근방으로 판단할 영역 크기 (픽셀 단위)
CONFIDENCE_THRESHOLD = 0.5  # 감지된 confidence의 임계값
ALERT_CONFIDENCE_THRESHOLD = 5  # 알람을 ON으로 간주하기 위한 최소 confidence 합계

# History to keep track of box occurrences
box_history = deque(maxlen=MAX_HISTORY)
confidence_history = deque(maxlen=MAX_HISTORY)

# Function to detect box (stub function, replace with real detection logic)
def detect_box():
    # 예시: 랜덤한 박스와 confidence를 생성
    h, w = 480, 640 # frame.shape
    box = (np.random.randint(0, w-100), np.random.randint(0, h-100),
           np.random.randint(100, w), np.random.randint(100, h))
    confidence = np.random.random()
    return box, confidence

# 현재 발생 box의 좌표가 Queue내 저장된 box중 하나라도 겹치는 것이 있으면 True리턴
def is_within_alert_area(box, history, alert_window):
    for past_box in history:
        if past_box:
            px1, py1, px2, py2 = past_box
            bx1, by1, bx2, by2 = box
            if (abs(px1 - bx1) < alert_window and abs(py1 - by1) < alert_window and
                abs(px2 - bx2) < alert_window and abs(py2 - by2) < alert_window):
                return True
    return False


alarm_on = False
alarm_counter = 0


for ii in range(100):

    # box와 box의 confidence는 이미지당 1개만 생성됨
    box, confidence = detect_box()
    
    if confidence > CONFIDENCE_THRESHOLD:
        # 기록된 히스토리 중 근방에서의 발생 빈도와 confidence 확인
        # Queue내의 저장영역들과 하나라도 겹침이 있으면 confidence더함(증가)
        if is_within_alert_area(box, box_history, ALERT_WINDOW):
            alarm_counter += confidence  # confidence 값을 누적
        else:  # 아니면 -1 또는 -0.1 등 적당한 값을 감소시킴 
            alarm_counter = max(0, alarm_counter - 0.1)
    else:
        alarm_counter = max(0, alarm_counter - 0.1)

    # History 업데이트
    box_history.append(box)
    confidence_history.append(confidence)

    # 알람 상태 결정
    if alarm_counter > ALERT_CONFIDENCE_THRESHOLD:
        alarm_on = True
    else:
        alarm_on = False

    print(f"프레임: {ii:3d}, 알람 값: {alarm_counter:6.3f}, 알람: {alarm_on}")

프레임:   0, 알람 값:  0.000, 알람: False
프레임:   1, 알람 값:  0.000, 알람: False
프레임:   2, 알람 값:  0.000, 알람: False
프레임:   3, 알람 값:  0.000, 알람: False
프레임:   4, 알람 값:  0.000, 알람: False
프레임:   5, 알람 값:  0.000, 알람: False
프레임:   6, 알람 값:  0.000, 알람: False
프레임:   7, 알람 값:  0.000, 알람: False
프레임:   8, 알람 값:  0.000, 알람: False
프레임:   9, 알람 값:  0.000, 알람: False
프레임:  10, 알람 값:  0.000, 알람: False
프레임:  11, 알람 값:  0.000, 알람: False
프레임:  12, 알람 값:  0.000, 알람: False
프레임:  13, 알람 값:  0.000, 알람: False
프레임:  14, 알람 값:  0.000, 알람: False
프레임:  15, 알람 값:  0.000, 알람: False
프레임:  16, 알람 값:  0.000, 알람: False
프레임:  17, 알람 값:  0.000, 알람: False
프레임:  18, 알람 값:  0.000, 알람: False
프레임:  19, 알람 값:  0.000, 알람: False
프레임:  20, 알람 값:  0.000, 알람: False
프레임:  21, 알람 값:  0.000, 알람: False
프레임:  22, 알람 값:  0.936, 알람: False
프레임:  23, 알람 값:  0.836, 알람: False
프레임:  24, 알람 값:  0.736, 알람: False
프레임:  25, 알람 값:  0.636, 알람: False
프레임:  26, 알람 값:  1.618, 알람: False
프레임:  27, 알람 값:  1.518, 알람: False
프레임:  28, 알람 값:  1.418, 알람: False
프레임:  29, 알람 값

In [50]:
import cv2
import numpy as np
from collections import deque, defaultdict
import pdb

# Parameters
ALERT_THRESHOLD = 20  # 특정 영역에서 Box가 나타난 횟수 및 confidence 합계의 임계값
MAX_HISTORY = 30  # 빈도를 추적할 프레임 수
ALERT_WINDOW = 100  # 근방으로 판단할 영역 크기 (픽셀 단위)
CONFIDENCE_THRESHOLD = 0.5  # 감지된 confidence의 임계값
ALERT_CONFIDENCE_THRESHOLD = 3  # 알람을 ON으로 간주하기 위한 최소 confidence 합계

# History to keep track of box occurrences
box_history = deque(maxlen=MAX_HISTORY)
confidence_history = deque(maxlen=MAX_HISTORY)

# 개별 박스의 알람 카운터를 저장할 딕셔너리
alarm_counters = defaultdict(float)

# Function to detect boxes (stub function, replace with real detection logic)
def detect_boxes():
    # 예시: 랜덤한 여러 박스와 confidence를 생성
    h, w = 480, 640 # frame.shape
    num_boxes = np.random.randint(1, 5)  # 랜덤한 개수의 박스 생성
    boxes = []
    confidences = []
    for _ in range(num_boxes):
        box = (np.random.randint(0, w-100), np.random.randint(0, h-100),
               np.random.randint(100, w), np.random.randint(100, h))
        confidence = np.random.random()
        boxes.append(box)
        confidences.append(confidence)
    return boxes, confidences

# 현재 발생한 box가 큐에 저장된 기존 box 중에서 겹치는 것을 찾고, 그 인덱스를 반환
def find_overlapping_index(box, history, alert_window):
    for i, past_box in enumerate(history):
        if past_box:
            px1, py1, px2, py2 = past_box
            bx1, by1, bx2, by2 = box
            if (abs(px1 - bx1) < alert_window and abs(py1 - by1) < alert_window and
                abs(px2 - bx2) < alert_window and abs(py2 - by2) < alert_window):
                return i  # 겹치는 박스의 인덱스 반환
    return None  # 겹치는 박스가 없으면 None 반환

for ii in range(100):

    # 박스와 박스의 confidence는 이미지당 여러 개 생성될 수 있음
    boxes, confidences = detect_boxes()
    
    for box, confidence in zip(boxes, confidences):
        if confidence > CONFIDENCE_THRESHOLD:
            # 큐에 저장된 기존 박스 중 겹치는 박스를 찾고 인덱스를 반환
            overlap_index = find_overlapping_index(box, box_history, ALERT_WINDOW)
            
            if overlap_index is not None:
                # 겹치는 박스가 있으면 해당 박스의 인덱스를 사용해 누적
                print(f"박스 {box}는 큐의 인덱스 {overlap_index}에 있는 박스와 겹침.")
                alarm_counters[overlap_index] += confidence  # 해당 인덱스의 alarm_counter를 누적
            else:
                # 겹치는 박스가 없으면 새로운 인덱스를 사용해 누적치 초기화
                new_index = len(box_history)
                alarm_counters[new_index] = max(0, alarm_counters[new_index] - 0.1)
        else:
            if overlap_index is not None:
                # confidence가 낮은 경우에도 겹치는 박스의 카운터를 감소시킴
                alarm_counters[overlap_index] = max(0, alarm_counters[overlap_index] - 0.1)

        # History 업데이트
        box_history.append(box)
        confidence_history.append(confidence)

    # 개별 알람 상태 결정
    overall_alarm_on = False
    triggered_index = None  # 알람이 발생한 인덱스를 추적
    
    for index, counter in alarm_counters.items():
        if counter > ALERT_CONFIDENCE_THRESHOLD:
            overall_alarm_on = True
            triggered_index = index  # 알람이 발생한 인덱스를 기록
            break

    if overall_alarm_on:
        # pdb.set_trace()
        formatted_alarm_counters = {k: f"{v:.3f}" for k, v in alarm_counters.items()}
        print(f"프레임: {ii:3d}, 알람 값: {dict(formatted_alarm_counters)}, 알람: {overall_alarm_on}, 트리거된 인덱스: {triggered_index}")
    else:
        formatted_alarm_counters = {k: f"{v:.3f}" for k, v in alarm_counters.items()}
        print(f"프레임: {ii:3d}, 알람 값: {dict(formatted_alarm_counters)}, 알람: {overall_alarm_on}")


프레임:   0, 알람 값: {}, 알람: False
프레임:   1, 알람 값: {4: '0.000', 5: '0.000', 7: '0.000'}, 알람: False
프레임:   2, 알람 값: {4: '0.000', 5: '0.000', 7: '0.000', 10: '0.000'}, 알람: False
프레임:   3, 알람 값: {4: '0.000', 5: '0.000', 7: '0.000', 10: '0.000', 13: '0.000'}, 알람: False
프레임:   4, 알람 값: {4: '0.000', 5: '0.000', 7: '0.000', 10: '0.000', 13: '0.000'}, 알람: False
프레임:   5, 알람 값: {4: '0.000', 5: '0.000', 7: '0.000', 10: '0.000', 13: '0.000', 16: '0.000', 17: '0.000', 18: '0.000', 19: '0.000'}, 알람: False
박스 (349, 281, 465, 242)는 큐의 인덱스 18에 있는 박스와 겹침.
프레임:   6, 알람 값: {4: '0.000', 5: '0.000', 7: '0.000', 10: '0.000', 13: '0.000', 16: '0.000', 17: '0.000', 18: '0.837', 19: '0.000'}, 알람: False
프레임:   7, 알람 값: {4: '0.000', 5: '0.000', 7: '0.000', 10: '0.000', 13: '0.000', 16: '0.000', 17: '0.000', 18: '0.737', 19: '0.000', 25: '0.000'}, 알람: False
박스 (310, 85, 438, 412)는 큐의 인덱스 6에 있는 박스와 겹침.
프레임:   8, 알람 값: {4: '0.000', 5: '0.000', 7: '0.000', 10: '0.000', 13: '0.000', 16: '0.000', 17: '0.000', 18: '0.737', 